In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
import sys
import os
import openai
import pandas as pd

# Add the path to the constants file to the system path
sys.path.append('../../')
from constants import *
from user_utils import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

# Get the current directory of the notebook
current_dir = os.path.dirname(os.path.abspath("../../data/amazon-beauty/rating_prediction.ipynb"))
print(f"current directory: {current_dir}")
# Construct the path to data file
data_path = os.path.join(current_dir, 'large_merged_data.csv')
print(f'data path: {data_path}')

current directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty
data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/data/amazon-beauty/large_merged_data.csv


In [6]:
# Read the data
amazon_data = pd.read_csv(data_path)
# get sample data of NUM_SAMPLES rows
amazon_data.info()
# get necessary columns
df = amazon_data[['title', 'rating', 'reviewText', 'reviewerID', 'category']]
df.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9767 entries, 0 to 9766
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      9767 non-null   float64
 1   reviewerID  9767 non-null   object 
 2   asin        9767 non-null   object 
 3   reviewText  9759 non-null   object 
 4   summary     9759 non-null   object 
 5   category    9767 non-null   object 
 6   title       9767 non-null   object 
dtypes: float64(1), object(6)
memory usage: 534.3+ KB


,title,rating,reviewText,reviewerID,category
0,Jenna Jameson Heartbreaker Perfume for women 3...,1.0,"I use a lot of perfume, I go through a new bot...",A2RYSCZOPEXOCQ,[]
1,Norelco 6885XL Deluxe Quadra Action Cord/Cordl...,5.0,"First, a little background. I've switched bet...",A141OPVE376YFI,[]
2,Norelco 6885XL Deluxe Quadra Action Cord/Cordl...,5.0,"First, a little background. I've switched bet...",A141OPVE376YFI,[]
3,Philips Norelco HQ5 Shaving Heads,5.0,I've had many Norelco razors in my 50 years of...,A1TVTDKNMSQ7XU,[]
4,Philips Norelco HQ5 Shaving Heads,5.0,I've had many Norelco razors in my 50 years of...,A1TVTDKNMSQ7XU,[]


In [9]:
# Split data such that for each user in the test set, we only retain a given number of rated items.
train_df_given_10, test_df_given_10 = split_data_by_rated_items(df, user_col='reviewerID', test_size=0.2, given_n=10, random_state=RANDOM_STATE)

# Split data based on percentages of unique users.
# Here, for example, 5% of users will be in the first set, next 10% in the second set, next 40% in the third set, and the rest in the test set.
sets = split_data_by_user_percentage(df, user_col='reviewerID', percentages=[0, 0.05, 0.15, 0.55, 1], random_state=RANDOM_STATE)
M5_pct_df, M10_pct_df, M40_pct_df, test_df = sets

# Leave-one-out split.
train_df, test_df_all_but_one = all_but_one(df, user_col='reviewerID', random_state=RANDOM_STATE)

# Printing the shapes of the data splits
print(f"train_df_given_10: {train_df_given_10.shape}")
print(f"test_df_given_10: {test_df_given_10.shape}")

print(f"M5_pct_df: {M5_pct_df.shape}")
print(f"M10_pct_df: {M10_pct_df.shape}")
print(f"M40_pct_df: {M40_pct_df.shape}")
print(f"test_df: {test_df.shape}")


train_df_given_10: (7813, 5)
test_df_given_10: (1954, 5)
M5_pct_df: (488, 5)
M10_pct_df: (960, 5)
M40_pct_df: (3956, 5)
test_df: (4363, 5)
